In [1]:
using CSV
using DataFrames
using Hungarian
using InferOpt
using Flux
using UnicodePlots
using StatsBase
using MLJ
using ProgressMeter
using Distances
using Random

In [2]:
using JuMP
using MathOptInterface
using Gurobi
using Graphs
using Plots
using JLD2
using FileIO

In [3]:
include("assingment_Util.jl")

display_solution_old (generic function with 1 method)

In [4]:
ENV["COLUMNS"] = 100
ENV["LINES"] = 100
Random.seed!(67);

In [5]:
file = File(format"JLD2", "Data/train_data.jld2")
train_data = JLD2.load(file)

Dict{String, Any} with 4 entries:
  "Y"   => [[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], [0 0 … 0 0; 0 0 … 0 1; … ; 0 0 … 0 0;…
  "X"   => Array{Float32, 3}[[0.0 1.0 … 1.0 0.0; 0.163556 0.859844 … 0.297643 0.799328; 45.0 14.0 ……
  "X_A" => DataFrame[46×9 DataFrame…
  "X_C" => DataFrame[47×9 DataFrame…

In [6]:
file = File(format"JLD2", "Data/test_data.jld2")
test_data = JLD2.load(file)

Dict{String, Any} with 4 entries:
  "Y"   => [[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 1; 0 0 … 0 0], [0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0;…
  "X"   => Array{Float32, 3}[[1.0 1.0 … 0.0 0.0; 0.209059 0.375959 … 0.321739 0.205797; 41.0 23.0 ……
  "X_A" => DataFrame[70×9 DataFrame…
  "X_C" => DataFrame[120×9 DataFrame…

In [7]:
X_train, X_A_train, X_C_train, Y_train = train_data["X"], train_data["X_A"], train_data["X_C"], train_data["Y"];

In [8]:
X_test, X_A_test, X_C_test, Y_test = test_data["X"], test_data["X_A"], test_data["X_C"], test_data["Y"];

In [9]:
"""
optimizer(cost_matrix)

Wrapper function for the hungarian assignment algorithm.
"""
function optimizer(cost_matrix)
    vector = hungarian(-cost_matrix)[1]
    n, m = size(cost_matrix)

    # Create a matrix of size n×n filled with 0's
    matrix = zeros(Int, n, m)

    # Set the value to 1 at the specified points in the vector
    for i in 1:length(vector)
        matrix[i, vector[i]] = 1
    end
    return matrix
end

optimizer

In [10]:
optimizer_layer = regularized = RegularizedGeneric(
    optimizer;
    omega=y -> half_square_norm(y),
    omega_grad=y -> y
    )
loss = FenchelYoungLoss(optimizer_layer);

In [11]:
og_encoder = Chain(
  Dense(4, 1),
  z -> dropdims(z; dims=1),
)

Chain(
  Dense(4 => 1),                        # 5 parameters
  var"#54#55"(),
) 

In [12]:
# Create a copy of the original encoder
encoder = deepcopy(og_encoder)

# Initialize the optimizer
opt_state = Flux.setup(Adam(), encoder)

# Initialize the losses array
losses = Float64[]
best_loss = Inf
best_model = deepcopy(encoder)

# Define the early stopping criterion
es = Flux.early_stopping(() -> best_loss, 5)

@showprogress 30 "Training model..." for epoch in 1:100
    l = 0.0
    # Iterate through each data point
    for (x, y) in zip(X_train, Y_train)
        # Calculate gradients using the current data point
        val, grads = Flux.withgradient(encoder) do m
            # Any code inside here is differentiated.
            # Evaluation of the model and loss must be inside!
            result = m(x)
            loss(result, y)
        end
        l+= val
        # Update the model parameters using the calculated gradients
        Flux.update!(opt_state, encoder, grads[1])
    end

    # Calculate average loss for this epoch
    average_loss = l / length(X_train)

    # Check for improvement 
    if average_loss < best_loss && average_loss >= 0
        best_loss = average_loss
        best_model = deepcopy(encoder)
    else
        if es()
            break
        end
    end

    # Append the average loss to the losses array
    push!(losses, average_loss)
end
encoder = best_model  # set encoder to the best model


Training model...   2%|█                                 |  ETA: 0:40:29

Training model... 100%|██████████████████████████████████| Time: 0:01:13


Chain(
  Dense(4 => 1),                        # 5 parameters
  var"#54#55"(),
) 

In [13]:
lineplot(losses; xlabel="Epoch", ylabel="Loss")

            ┌────────────────────────────────────────┐ 
          0 │⠑⢄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
            │⠀⠀⠈⠢⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
            │⠀⠀⠀⠀⠈⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
            │⠀⠀⠀⠀⠀⠀⠀⠑⠢⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
            │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠒⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
            │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
            │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠢⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
   Loss     │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠢⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
            │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠢⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
            │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠢⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
            │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠒⢄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
            │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠑⠢⣀⠀⠀⠀⠀⠀⠀⠀│ 
            │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠢⢄⡀⠀⠀⠀│ 
            │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠒⠤⡀│ 
        -50 │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈│ 
            └────────────────────────────────────────┘ 
            ⠀1⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀

In [14]:
Y_test_pred = [optimizer(encoder(x)) for x in X_test];

In [16]:
pred_deliveries, pred_constraints = create_deliveries(X_A_test, X_C_test, Y_test_pred)
true_deliveries, true_constraints = create_deliveries(X_A_test, X_C_test, Y_test)

(Any[Any[Delivery(5.126374877523316, 4.406390187627437), Delivery(5.1204729157687865, 4.334089906634947), Delivery(5.036515647493489, 3.6036883394790844), Delivery(5.051932414400978, 3.5303819557872007), Delivery(5.1303406268326714, 4.296859465636211), Delivery(5.1204729157687865, 4.334089906634947), Delivery(5.132955076513357, 4.639858420562722), Delivery(5.134041500871803, 4.598190619749294), Delivery(5.081140133034815, 4.121061264108667), Delivery(5.07584980205084, 3.7626139343616054)  …  Delivery(4.9126008409149176, 4.209983181019458), Delivery(4.913548990898646, 4.175628404514657), Delivery(4.895501841898249, 4.099008184338153), Delivery(4.902555315701303, 3.9596151888925597), Delivery(4.935536208681828, 4.061245525564403), Delivery(4.884667714272846, 3.965063684612284), Delivery(5.013164114122896, 4.346298301241377), Delivery(5.008799200904464, 4.314894454882154), Delivery(5.026286056693232, 4.401612044016113), Delivery(5.025902115999137, 4.417031359969121)], Any[Delivery(4.96763

In [17]:
pred_solution = []
@showprogress 30 "progress ..." for (delivery, constraint) in zip(pred_deliveries, pred_constraints)
   push!(pred_solution, solve_tsp(delivery, constraint, Gurobi.Optimizer));
end

progress ...   2%|█                                      |  ETA: 1:00:09

Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-28
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-28
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-28


progress ...   2%|█                                      |  ETA: 1:24:37

Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-28


progress ...   4%|██                                     |  ETA: 1:41:52

...

Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-28
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-28


progress ...  97%|██████████████████████████████████████ |  ETA: 0:05:17

Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-28
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-28
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-28


progress ... 100%|███████████████████████████████████████| Time: 3:11:51


Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-28
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-28
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-28
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-28
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-28


In [18]:
file = File(format"JLD2", "Data/regulized_output.jld2")
  
# Save data into the file
save(file, "delivery", pred_deliveries, "constraint", pred_constraints, "solutions", pred_solution)